In [29]:
import pandas as pd
import pandahouse

import seaborn as sns

from utils.tags import tag_elements

__Загрузка датафрейма__

In [2]:
# pandahouse не поддерживает тип Array, поэтому в pandas необходимо
# будет выполнить некоторые преобразования.

SQL = '''

SELECT article_id,
       date,
       rate,
       views,
       bookmarks,
       comments,
       header,
       toString(tags) AS tags

FROM articles

'''


article_data = pandahouse.read_clickhouse(query=SQL)

__Предобработка данных__

_Предобработка тэгов_

In [38]:
# Теги загружены в виде строки вида ['tag_1', 'tag_1', 'tag_1'],
# привожу его к массиву.

article_data.tags = [tag for tag in article_data.tags.str.replace(
                     pat="[\[\]']", repl='', regex=True).str.split(',') if tag != '']

# Раскрываю датафрейм по колонке tags. Применяю к каждому отдельному тегу tag_elements.

article_data = article_data.explode('tags')
article_data.tags = article_data.tags.apply(tag_elements)

In [39]:
# Выбираю тэги и подсчитываю их встречаемость.

tags = article_data.groupby('tags', as_index=False) \
                   .agg({'comments': 'count'}) \
                   .rename(columns={'tags': 'tag', 'comments': 'used'}) \
                   .sort_values(['used', 'tag'])

min_tag_used = 100
max_tag_used = 10_000

# Удаляю редко используемые "мусорные" тэги
tags.drop(tags.query('used <= @min_tag_used').index, inplace=True)
tags.drop(tags.query('used >= @max_tag_used').index, inplace=True)

# Удаляю пустые тэги
tags.drop(tags.query('tag == ""').index, inplace=True)

# Определяю язык тэга и удаляю теги на двух языках
tags.drop(tags[(tags.tag.str.contains('^[а-я]+$', regex=True) == False) & (tags.tag.str.contains('^[a-z]+$', regex=True) == False)].index, inplace=True)

In [40]:
# Сохраняю теги для их отбора
tags.to_csv('./datasets/tags.csv', index=False)

In [6]:
# Загружаю отобранные теги
tags = pd.read_csv('./datasets/tags.csv')

In [7]:
# Совмещаю данные по тэгам с исходным датафреймом.

article_data = article_data.merge(tags, left_on='tags', right_on='tag', how='left') \
                           .drop(columns='tags') \
                           .dropna()